In [1]:
# Imports
## if this doesn't work, go to https://brew.sh/ and run the installation command in terminal
## and then run
## conda install -c conda-forge selenium
#### isbnlib will also need to be installed:
## pip install isbnlib
## occasionally run in terminal:
## brew update && brew doctor 

import os

import csv

from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup

# isbnlib is not needed for this notebook
# from isbnlib import is_isbn10, is_isbn13, clean

In [2]:
# Statics
# Note: this URL works, but could probably be cleaned up
# Note2: The source of titles to be searchedshould be a text document, 
# formatted with one title per line.
# It should be free of typos, and should record titles exactly as they appear
# on the title page of the book. 

base_url = "http://bobcat.library.nyu.edu/primo_library/libweb/action/"
base_url += "search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=all&indx=1&dum=true&srt=rank&vid=NYU&frbg=&vl%28freeText0%29="

infile = "data/title-examples.txt"

In [3]:
# Add exact-search characters to each line
append_text = "%22"
with open(infile, "r") as f:
    buff = []
    for line in f:
        if line.strip():
             buff.append("%s%s%s"% (append_text,line.strip(),append_text))
    updatedNames=open("data/title-examples2.txt",'w')
    updatedNames.write("\n".join(buff))
    updatedNames.close()

In [4]:
# Further formatting of titles into a searchable form
infile2 = "data/title-examples2.txt"
with open(infile2, "r") as f:
    titles = f.read().replace(" ", "+").splitlines()

In [5]:
# Create browser instance
# Note: this requires Firefox to be installed

browser = webdriver.Firefox(executable_path="/usr/local/bin/geckodriver")

In [8]:
# Function for finding title matches in Bobcat (via Selenium)

def check_bobcat_title(title):
    #check_bobcat = False # Set default return
    
    url = base_url + title # Build URL string
    
    browser.get(url) # Open url in browser instance; should trap response errors
    alert = browser.find_elements_by_class_name('alert') # Only missing titles have an class called alert
    
    return True if len(alert) == 0 else False

    

In [11]:
# Iterate over isbns and find matches

matches = []

for title in titles:
    match = check_bobcat_title(title)
    temp = title.replace("%22", "").replace("+", " ").title()
    matches.append((temp, match))

In [12]:
# Close browser instance

browser.quit()

In [13]:
# Export to csv

with open('matches.csv','w') as out:
    csv_out=csv.writer(out, quotechar = "'")
    csv_out.writerow(['title','match'])
    for row in matches:
        csv_out.writerow(row)